# 2nd Requirement

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [25]:
file_name = "50EA1DAA6NA002459-1 (1).csv"
# file_name = "50EA1DAA8NA002849-FS.xlsx"
# file_name = "50EA1DAA8NA002849-FS (1).xls"
file_name

'50EA1DAA6NA002459-1 (1).csv'

In [26]:
df = pd.read_csv(file_name)
# df = pd.read_excel(file_name)

df['new_timestamp'] = df['@timestamp'].apply(lambda x: x.replace("@","").replace("+\s","").strip())
df['new_timestamp'] = pd.to_datetime(df['new_timestamp'])

# reverse the dataset
df = df.iloc[::-1]

print("Actual: ")
print(df.shape)

print("After Filter: ")
df = df.loc[df.ident == 'FLASH_SERVICE',['new_timestamp','message']]
df.reset_index(inplace=True,drop=True)

print(df.shape)
df.head()

Actual: 
(101448, 16)
After Filter: 
(93987, 2)


,new_timestamp,message
0,2022-01-25 18:20:27,"num_bytes: 9, msg_type: 6"
1,2022-01-25 18:20:27,DOIP_TYPE response
2,2022-01-25 18:20:27,In the routing activation request
3,2022-01-25 18:20:27,"num_bytes: 9, msg_type: 6"
4,2022-01-25 18:20:27,DOIP_TYPE response


In [27]:
ip_connection_string = "Successfully connected to"

ip_df = df.loc[df.message.str.contains(ip_connection_string),['new_timestamp', 'message']]

ip_df.reset_index(drop=True, inplace=True)
ip_df.rename(columns={'message':'connect_msg','new_timestamp':'connect_ts'},inplace=True)

ip_df['connect_msg_ip'] = ip_df['connect_msg'].apply(lambda x: x.replace(ip_connection_string,"").strip())

ip_df['ip_endswith'] = ip_df.connect_msg.apply(lambda x: x.split(".")[-1])

ip_df.sort_values(['ip_endswith','connect_ts'], ascending=[True,False], inplace=True, ignore_index=True)

ip_df['rank'] = ip_df.groupby(['ip_endswith']).cumcount() + 1

ip_df = ip_df.sort_values(['connect_msg', 'connect_ts']).drop_duplicates('connect_msg', keep='last')

ip_df.reset_index(drop = True, inplace = True)

print(ip_df.shape)
ip_df

(10, 5)


,connect_ts,connect_msg,connect_msg_ip,ip_endswith,rank
0,2022-01-25 18:20:27,Successfully connected to 192.168.1.1,192.168.1.1,1,1
1,2022-01-25 18:20:27,Successfully connected to 192.168.1.12,192.168.1.12,12,1
2,2022-01-25 18:20:27,Successfully connected to 192.168.1.2,192.168.1.2,2,1
3,2022-01-25 18:20:27,Successfully connected to 192.168.1.3,192.168.1.3,3,1
4,2022-01-25 18:23:20,Successfully connected to 192.168.1.4,192.168.1.4,4,1
5,2022-01-25 18:20:27,Successfully connected to 192.168.1.5,192.168.1.5,5,1
6,2022-01-25 18:20:27,Successfully connected to 192.168.1.6,192.168.1.6,6,1
7,2022-01-25 18:20:27,Successfully connected to 192.168.1.7,192.168.1.7,7,1
8,2022-01-25 18:36:27,Successfully connected to 192.168.1.71,192.168.1.71,71,1
9,2022-01-25 18:20:27,Successfully connected to 192.168.1.9,192.168.1.9,9,1


In [28]:
a = df.loc[df.message.str.contains("response: read_bytes:")]
a = a.loc[a.message.str.startswith("[")]
req_resp_indexs = a.index.tolist()
print(len(req_resp_indexs))

14


In [29]:
e = []
for i in req_resp_indexs:
    p = df.iloc[i-1]['message']
    b = p.split()[-6]
    c = df.iloc[i]['message']
    msg_ts = df.iloc[i]['new_timestamp']
    hex_ts = df.iloc[i-1]['new_timestamp']
    d = str(int(b, 16))
    f = {'response_val': c,'resp_sub': c[:3], 'Hex Code': b,
         'ip_endswith': d, 'msg_ts': msg_ts, 'hex_ts':hex_ts,'code':p}
    e.append(f)
    # print("Response: ",c,"Hex String Code:",b, "IP Address: ",d)
    
g = pd.DataFrame(e)

g.sort_values(by=['ip_endswith','msg_ts'],ascending=[True,False], inplace=True, ignore_index=True)
g['rank'] = g.groupby(['ip_endswith']).cumcount() + 1

g = g.sort_values(by=['resp_sub','msg_ts']).drop_duplicates('resp_sub',keep='first')
g.reset_index(drop=True, inplace=True)

print(g.shape)
g

(10, 8)


,response_val,resp_sub,Hex Code,ip_endswith,msg_ts,hex_ts,code,rank
0,[0] response: read_bytes: 17,[0],01,1,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 01 10 00 00 0...,1
1,[1] response: read_bytes: 17,[1],02,2,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 02 10 00 00 0...,1
2,[2] response: read_bytes: 17,[2],03,3,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 03 10 00 00 0...,1
3,[3] response: read_bytes: 17,[3],04,4,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 04 10 00 00 0...,3
4,[4] response: read_bytes: 17,[4],05,5,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 05 10 00 00 0...,1
5,[5] response: read_bytes: 17,[5],06,6,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 06 10 00 00 0...,1
6,[6] response: read_bytes: 17,[6],07,7,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 07 10 00 00 0...,1
7,[7] response: read_bytes: 17,[7],09,9,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 09 10 00 00 0...,1
8,[8] response: read_bytes: 17,[8],47,71,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 47 10 00 00 0...,3
9,[9] response: read_bytes: 17,[9],0c,12,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 0c 10 00 00 0...,1


In [30]:
h = pd.merge(ip_df,g, on=['ip_endswith'])
print(h.shape)
h

(10, 12)


,connect_ts,connect_msg,connect_msg_ip,ip_endswith,rank_x,response_val,resp_sub,Hex Code,msg_ts,hex_ts,code,rank_y
0,2022-01-25 18:20:27,Successfully connected to 192.168.1.1,192.168.1.1,1,1,[0] response: read_bytes: 17,[0],01,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 01 10 00 00 0...,1
1,2022-01-25 18:20:27,Successfully connected to 192.168.1.12,192.168.1.12,12,1,[9] response: read_bytes: 17,[9],0c,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 0c 10 00 00 0...,1
2,2022-01-25 18:20:27,Successfully connected to 192.168.1.2,192.168.1.2,2,1,[1] response: read_bytes: 17,[1],02,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 02 10 00 00 0...,1
3,2022-01-25 18:20:27,Successfully connected to 192.168.1.3,192.168.1.3,3,1,[2] response: read_bytes: 17,[2],03,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 03 10 00 00 0...,1
4,2022-01-25 18:23:20,Successfully connected to 192.168.1.4,192.168.1.4,4,1,[3] response: read_bytes: 17,[3],04,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 04 10 00 00 0...,3
5,2022-01-25 18:20:27,Successfully connected to 192.168.1.5,192.168.1.5,5,1,[4] response: read_bytes: 17,[4],05,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 05 10 00 00 0...,1
6,2022-01-25 18:20:27,Successfully connected to 192.168.1.6,192.168.1.6,6,1,[5] response: read_bytes: 17,[5],06,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 06 10 00 00 0...,1
7,2022-01-25 18:20:27,Successfully connected to 192.168.1.7,192.168.1.7,7,1,[6] response: read_bytes: 17,[6],07,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 07 10 00 00 0...,1
8,2022-01-25 18:36:27,Successfully connected to 192.168.1.71,192.168.1.71,71,1,[8] response: read_bytes: 17,[8],47,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 47 10 00 00 0...,3
9,2022-01-25 18:20:27,Successfully connected to 192.168.1.9,192.168.1.9,9,1,[7] response: read_bytes: 17,[7],09,2022-01-25 18:20:27,2022-01-25 18:20:27,02 fd 00 06 00 00 00 09 0e 00 00 09 10 00 00 0...,1


In [31]:
p = h[['connect_ts','connect_msg','response_val','code']].copy()
p.to_csv("output.csv",index=False)

In [32]:
alfa = h.drop_duplicates('ip_endswith').set_index('resp_sub')['connect_msg_ip'].to_dict()
beta = h.drop_duplicates('ip_endswith').set_index('connect_msg_ip')['resp_sub'].to_dict()

print("Hex to IP Mapping....")
print(alfa)

print("\n","*"*10,"\n")

print("IP to Hex Mapping....")
print(beta)

Hex to IP Mapping....
{'[0]': '192.168.1.1', '[9]': '192.168.1.12', '[1]': '192.168.1.2', '[2]': '192.168.1.3', '[3]': '192.168.1.4', '[4]': '192.168.1.5', '[5]': '192.168.1.6', '[6]': '192.168.1.7', '[8]': '192.168.1.71', '[7]': '192.168.1.9'}

 ********** 

IP to Hex Mapping....
{'192.168.1.1': '[0]', '192.168.1.12': '[9]', '192.168.1.2': '[1]', '192.168.1.3': '[2]', '192.168.1.4': '[3]', '192.168.1.5': '[4]', '192.168.1.6': '[5]', '192.168.1.7': '[6]', '192.168.1.71': '[8]', '192.168.1.9': '[7]'}


# 2nd Step

In [9]:
selected_ip = '192.168.1.1'

selected_bracket = beta[selected_ip]
selected_ip, selected_bracket

('192.168.1.1', '[0]')

In [19]:
a1 = df[df.message.str.contains("tester request bytes:")]
a1['message_key'] = a1.message.apply(lambda x: x[:3])
a1['sort_key'] = 1
a1.reset_index(inplace=True, drop=True)
print(a1.shape)

b1 = df[df.message.str.contains("tester response ")]
b1['message_key'] = b1.message.apply(lambda x: x[:3])
b1['sort_key'] = 2
b1.reset_index(inplace=True, drop=True)
print(b1.shape)

c1 = pd.concat([a1,b1])
c1

(3555, 4)
(3553, 4)


,new_timestamp,message,message_key,sort_key
0,2022-01-25 18:20:27,[3] tester request bytes: 14,[3],1
1,2022-01-25 18:20:27,[8] tester request bytes: 14,[8],1
2,2022-01-25 18:20:27,[9] tester request bytes: 14,[9],1
3,2022-01-25 18:20:27,[6] tester request bytes: 14,[6],1
4,2022-01-25 18:20:27,[5] tester request bytes: 14,[5],1
...,...,...,...,...
3548,2022-01-25 18:38:03,[1] tester response read bytes: 15,[1],2
3549,2022-01-25 18:38:03,[4] tester response read bytes: 15,[4],2
3550,2022-01-25 18:38:04,[0] tester response read bytes: -1,[0],2
3551,2022-01-25 18:38:04,[8] tester response read bytes: 15,[8],2


In [20]:
d1 = c1[c1.message_key == selected_bracket]
d1.sort_values(by=['nerequest_response_dfrequest_response_dfw_timestamp','sort_key'],inplace=True,ignore_index=True)
d1['print_col'] = d1['message'] +"   "+ d1['new_timestamp'].astype(str)
print(d1.shape)
d1.head()

(810, 5)


,new_timestamp,message,message_key,sort_key,print_col
0,2022-01-25 18:20:27,[0] tester request bytes: 14,[0],1,[0] tester request bytes: 14 2022-01-25 18:2...
1,2022-01-25 18:20:28,[0] tester response read bytes: 15,[0],2,[0] tester response read bytes: 15 2022-01-2...
2,2022-01-25 18:20:31,[0] tester request bytes: 14,[0],1,[0] tester request bytes: 14 2022-01-25 18:2...
3,2022-01-25 18:20:31,[0] tester response read bytes: 15,[0],2,[0] tester response read bytes: 15 2022-01-2...
4,2022-01-25 18:20:34,[0] tester request bytes: 14,[0],1,[0] tester request bytes: 14 2022-01-25 18:2...


In [21]:
d1['print_col']

0      [0] tester request bytes: 14   2022-01-25 18:2...
1      [0] tester response read bytes: 15   2022-01-2...
2      [0] tester request bytes: 14   2022-01-25 18:2...
3      [0] tester response read bytes: 15   2022-01-2...
4      [0] tester request bytes: 14   2022-01-25 18:2...
                             ...                        
805    [0] tester response read bytes: 15   2022-01-2...
806    [0] tester request bytes: 14   2022-01-25 18:3...
807    [0] tester response read bytes: 15   2022-01-2...
808    [0] tester request bytes: 14   2022-01-25 18:3...
809    [0] tester response read bytes: -1   2022-01-2...
Name: print_col, Length: 810, dtype: object

In [43]:
writer = pd.ExcelWriter("abc.xlsx")
a.to_excel(writer, "summary", index=False)
# worksheet = writer.sheets['summary']
# worksheet.set_column(0,10, 18)
writer.save()
writer.close()

,new_timestamp,message
63,2022-01-25 18:20:27,[9] response: read_bytes: 17
70,2022-01-25 18:20:27,[8] response: read_bytes: 17
77,2022-01-25 18:20:27,[7] response: read_bytes: 17
84,2022-01-25 18:20:27,[6] response: read_bytes: 17
91,2022-01-25 18:20:27,[5] response: read_bytes: 17
98,2022-01-25 18:20:27,[4] response: read_bytes: 17
105,2022-01-25 18:20:27,[3] response: read_bytes: 17
112,2022-01-25 18:20:27,[2] response: read_bytes: 17
119,2022-01-25 18:20:27,[1] response: read_bytes: 17
126,2022-01-25 18:20:27,[0] response: read_bytes: 17
